In [2]:
import torch
import numpy as np
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torchvision.models import resnet50
import os
from torchvision.models.resnet import ResNet50_Weights

# Определение модели
class SimpleResNet(nn.Module):
    def __init__(self, num_classes):
        super().__init__()
        self.base_model = resnet50(weights=ResNet50_Weights.IMAGENET1K_V2)
        num_ftrs = self.base_model.fc.in_features
        self.base_model.fc = nn.Linear(num_ftrs, num_classes)

    def forward(self, x):
        return self.base_model(x)


num_classes = 15
device = torch.device("cuda")

model = SimpleResNet(num_classes)
model.load_state_dict(torch.load("C:/Users/user/Desktop/госкатлог/best_model-8b-256picV2.pth"))
model.base_model.fc = nn.Identity()
model = model.to(device)
model.eval()

# Подготовка DataLoader
transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor(),
    #transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])
dataset_path = r"C:/Users/user/Desktop/госкатлог/train_train_2"  # Путь к вашему датасету
dataset = datasets.ImageFolder(dataset_path, transform=transform)
dataloader = DataLoader(dataset, batch_size=32, shuffle=False)

# Извлечение эмбеддингов
def extract_embeddings(dataloader, model):
    model.eval()
    embeddings = []
    labels = []
    filenames = []  # Для сохранения имен файлов

    with torch.no_grad():
        for i, (images, targets) in enumerate(dataloader):
            images = images.to(device)
            output = model(images)
            embeddings.extend(output.cpu().numpy())
            # Получаем имена файлов для текущей партии
            batch_filenames = [os.path.basename(dataloader.dataset.samples[j][0]) for j in range(i * dataloader.batch_size, min((i + 1) * dataloader.batch_size, len(dataloader.dataset.samples)))]
            labels.extend(batch_filenames)  # Теперь labels будет содержать имена файлов
            filenames.extend(batch_filenames)

    return np.array(embeddings), labels, filenames

embeddings, labels, filenames = extract_embeddings(dataloader, model)

# Вывод размеров полученных массивов для проверки
print("Embeddings shape:", embeddings.shape)
print("Labels shape:", len(labels))
print("Number of filenames:", len(filenames))


Embeddings shape: (15831, 2048)
Labels shape: 15831
Number of filenames: 15831


In [3]:
import numpy as np

# Сохранение эмбеддингов и меток
np.save('saved_embeddings.npy', embeddings)
np.save('saved_labels.npy', labels)

In [4]:
import numpy as np

# Загрузка файлов с эмбеддингами и именами файлов
embeddings = np.load(r'saved_embeddings.npy')
filenames = np.load(r'saved_labels.npy')

# Вывод информации о содержимом файлов
print("Эмбеддинги:")
print(embeddings)

print("Имена файлов:")
print(filenames)


Эмбеддинги:
[[0.931576   0.15719768 0.12142767 ... 0.         1.2334096  0.01679719]
 [0.5906469  0.4499346  0.07321776 ... 0.         0.68063223 0.08775743]
 [0.07113336 0.05423784 0.19900262 ... 0.         0.37715548 0.03191456]
 ...
 [0.         0.         0.06797829 ... 0.11176834 0.02451316 0.        ]
 [0.03036493 0.00733673 0.         ... 0.         0.         0.00267379]
 [0.         0.         0.         ... 0.         0.         0.        ]]
Имена файлов:
['14246331.jpg' '14249267.jpg' '15000074.jpg' ... '7750119.jpg'
 '7750123.jpg' '7750125.jpg']
